In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

In [2]:
df  = pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace = True)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
#train test spliting 

In [7]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size = 0.3,random_state = 44)


In [8]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,male,54.0,0,0,51.8625,S
382,3,male,32.0,0,0,7.9250,S
724,1,male,27.0,1,0,53.1000,S
424,3,male,18.0,1,1,20.2125,S
250,3,male,NaN,0,0,7.2500,S


In [9]:
#imputation transformer
trf1 = ColumnTransformer([
    ("impute_age",SimpleImputer(),[2]),
    ("imputer_embarked",SimpleImputer(strategy = 'most_frequent'),[6])
],remainder = 'passthrough')#--> passthrough means dont drop another columns

In [10]:
#one hot encodeing
trf2 = ColumnTransformer([
    ("O.H.E_sex_embarked",OneHotEncoder(sparse=False,handle_unknown="ignore"),[1,6])
],remainder = "passthrough")

In [11]:
trf3 = ColumnTransformer([
    ("scale",MinMaxScaler(),slice(0,10))
])# slice uses for to apply MinMaxScaler to all the column to the range of (0,10)

In [12]:
# trf4 = SelectKBest(score_func=chi2,k=8)

In [13]:
trf5 = DecisionTreeClassifier()

# Make Pipeline


In [14]:
#Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf5)

In [15]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imputer_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('O.H.E_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [16]:
y_pred = pipe.predict(x_test)

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
accuracy_score(y_test,y_pred)*100

63.80597014925373

# Exporting the Pipeline

In [19]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))